## 4. A3C - data parallelism 
#### : 여러 자식 process가 환경과 상호작용하여 얻은 경험을 모아 하나의 main process가 학습 
<img src = "./image/data.png">

### 구현 
#### - 간단함과 효율성을 위해 학습 process로부터의 신경망 가중치 전파는 구현되지 않음 
- 직접 모으고 자식에게 가중치를 보내는 것 대신 모든 process의 네트워크는 공유됨 
    - PyTorch의 내장 기능 
    - 신경망 생성 시 **share_memory()** 를 호출하여 각 process의 모든 가중치에 같은 nn.Module 인스턴스를 사용
    
    - 0의 간접비를 가지는 것이 이 방법의 핵심, 성능 향상 가능 
        - CUDA) GPU 메모리는 전체 host의 process간에 공유됨
        - CPU) 프로세스 간 통신(IPC:Inter-Process Communication)을 사용할 경우 
             
    - **하지만** 
        - 예제 구현에서는 한 기계에서 하나의 GPU를 사용하여 학습하고 데이터를 모음 
        - Pong 예제에서는 크게 문제 없지만, 더 큰 확장성을 원한다면 네트워크 가중치를 공유하는 부분 확장 필요 
        
### 주요 클래스 및 함수 
#### - class Atari2C(nn.Module)
- Actor-Critic 신경망 
- 출력은 (policy, value) 튜플 

#### - class RewardTracker
- 전체(꽉 찬)(full) 에피소드의 할인된 보상 관리 
- Tensorboard에 기록 
- game 해결 여부 확인 

#### - unpack_batch(batch, net , last_val_gamma)
- n에피소드 step에 대한 transition 배치 (state, reward, action, last_state)를 학습에 적합하도록 변환 

https://www.pydoc.io/pypi/ptan-0.3/autoapi/agent/index.html?highlight=agent%20policyagent#agent.PolicyAgent

## A3C로 pong게임하기 구현 
- 이전 챕터들에서 봤던 코드와 거의 비슷 
- 자식 프로세스 추가 

In [2]:
import gym 
import ptan 
import numpy as np
import argparse
import collections
from tensorboardX import SummaryWriter

import torch.nn.utils as nn_utils
import torch.nn.functional as F
import torch.optim as optim
import torch.multiprocessing as mp #python의 multiproecessing 패키지가 아님 주의

from lib import common

In [4]:
GAMMA = 0.99
LEARNING_RATE = 0.001
ENTROPY_BETA = 0.01
BATCH_SIZE = 128

REWARD_STEPS = 4
CLIP_GRAD = 0.1

PROCESSES_COUNT = 4 #자식 process 개수 (CPU core개수)
#장치관리자 - 프로세스 확인가능 

NUM_ENVS = 15 #각 자식 process가 사용할 환경 개수 

#총 병렬 환경 개수 = 60 (15 * 4) 

In [6]:
"""환경 생성""" 
def make_env():
    return ptan.common.wrappers.wrap_dqn(gym.make(ENV_NAME)) 

### data_func (net, device, train_queue)
#### - 자식 process 에서 실행됨 
#### - train_queue 
- 자식 process가 master process로 보낼 데이터에 사용 
- 생산자가 여럿, 소비자가 하나일 때 큐를 사용 
- 두 가지 종류의 객체를 포함 
    - TotalReward: 완료된 에피소드의 할인된 보상인 소수값의 reward 필드만을 가진 객체 
    - ptan.experience.ExperienceSourceFirstLast()
        - REWARD_STEPS만큼의 연속에서 (첫번째 상태, 첫번째 상태에서 취한 액션, 보상, 마지막 상태)를 가지는 객체 
        - 학습에 사용 

In [ ]:
TotalReward = collections.namedtuple('TotalReward', field_names="reward")

"""총 에피소드 보상을 main 학습 process에 전달"""
def data_func(net, device, train_queue):
    envs = [make_env() for _ in range(NUM_ENVS)] #15 
    
    
    agent = ptan.agent.PolicyAgent(lambda x: net(x)[0], device=device, apply_softmax=True)
    #net(x)의 출력은 튜플 (policy, value) 
    #ptan.agent.PolicyAgent(model): 모델로부터 액션 확률분포를 얻고 그로부터 액션 선택 
    
    
    exp_source = ptan.experience.ExperienceSourceFirstLast(envs, agent, gamma=GAMMA, steps_count=REWARD_STEPS)
    #모든 trajectory 조각(전체 에피소드의 steps_count만큼 씩) 에 대해 할인된 보상 계산 
    #(첫번째 상태, 첫번째 상태에서 취한 액션, 보상, 마지막 상태) 반환 


    for exp in exp_source:
        new_rewards = exp_source.pop_total_rewards()##
        
        #보상이 0(False)이 아닐때 
        if new_rewards:
            train_queue.put(TotalReward(reward=np.mean(new_rewards))) 
            #한 에피소드 조각들의 총 보상들에 대한 평균을 저장 
        
        train_queue.put(exp) 
        #train_queue에 총보상평균, 경험 저장 

In [17]:
def why(a):
    k = []
    if a :
        k.append(1)
        
    k.append(2)
    
    return k

In [18]:
why(1)

[1, 2]

In [ ]:
if __name__ == "__main__":
    
    mp.set_start_method('spawn') #PyTorch에서는 best option 

    
    parser = argparse.ArgumentParser()
    parser.add_argument("--cuda", default=False, action="store_true", help="Enable cuda")
    parser.add_argument("-n", "--name", required=True, help="name of the run") 
    #tensorboard에 표시될 이름 
    args = parser.parse_args()
    device = "cuda" if args.cuda else "cpu"
    
    writer = SummaryWriter(comment="-a3c-data_" + NAME + "_" + args.name)
    
    
    """환경, 네트워크, 최적화 생성"""
    env = make_env()
    
    net = common.AtariA2C(env.observation_space.shape, env.action_space.n).to(device) #device로 이동 
    net.share_memory() #가중치 공유 요청 #GPU는 default로 지원, CPU는 사용 필요 
    
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, eps=1e-3)
    
    
    
    """자식 process가 메인 process에 전달할 데이터 큐"""
    train_queue = mp.Queue(maxsize=PROCESSES_COUNT) #꽉찬 큐에는 새로 입력 불가능 (for on-policy)
    data_proc_list = [] #data_process_list
    
    for _ in range(PROCESSES_COUNT): #4
        data_proc = mp.Process(target=data_func, args=(net, device, train_queue))
        #mp.Process(target, args): args를 인자로 target함수에 전달하여 실행
        #함수와 인자를 할당하여 Process 객체 생성 
        
        data_proc.start() #data_fun()이 자식 process에서 실행 #set_start_
        data_proc_list.append(data_proc) 
        
        #train_queue에 각 자식 process의 데이터 저장 
        
    
    """학습"""
    batch = []
    step_idx = 0
    
    
    try : 
        with common.RewardTracker(writer, stop_reward=REWARD_BOUND) as tracker:
            with ptan.common.utils.TBMeanTracker(writer, batch_size=100) as tb_tracker:
                
                while True:
                    train_entry = train_queue.get() #소비자 
                    
                    #queue에 있는게 reward
                    if isinstance(train_entry, TotalReward):
                        if tracker.reward(train_entry.reward, step_idx): 
                            #학습데이터의 보상이 REWARD_BOUND(18)보다 크면 사용 안함 
                            break
                            
                        continue #작으면 
                      
                    step_idx += 1 
                    batch.append(train_entry) #batch에는 exp객체가 저장 
                    
                    #batch가 쌓일때까지 
                    if len(batch) < BATCH_SIZE:
                        continue 
                        
                    
                    #batch가 다 쌓이면 
                    #gamma**4 
                    states_v, actions_t, vals_ref_v = common.unpack_batch(batch, net, last_val_gamma=GAMMA**REWARD_STEPS, device=device)
                    #unpack_batch(batch, net, last_val_gamma): 환경 transition batch를 3개의 tensor로 반환 
                                            ##(states batch, taken actions batch, Q-values batch)
                    
                    batch.clear()
                    
                    
                    """최적화"""
                    optimizer.zero_grad()
                    
                    logits_v, value_v = net(states_v) #(policy, value)
                    
                    loss_value_v = F.mse_loss(value_v.squeeze(-1), vals_ref_v)
                    
                    log_prob_v = F.log_softmax(logits_v, dim=1)
                    
                    
                    adv_v = vals_ref_v - value_v.detach()
                    
                    
                    log_prob_actions_v = adv_v * log_prob_v[range(BATCH_SIZE), actions_t]
                    
                    loss_policy_v = -log_prob_actions_v.mean()
                    
                    prob_v = F.softmax(logits_v, dim=1)
                    
                    entropy_loss_v = ENTROPY_BETA * (prob_v * log_prob_v).sum(dim=1).mean()
                    
                    loss_v = entropy_loss + loss_value_v + loss_policy_v 
                    loss_v.backward()
                    
                    nn_utils.clip_grad_norm_(net.parameters(), CLIP_GRAD)
                    
                    optimizer.step()
                    
                    """tensorboard"""
                    tb_tracker.track("advantage", adv_v, step_idx)
                    tb_tracker.track("values", value_v, step_idx)
                    tb_tracker.track("batch_rewards", vals_ref_v, step_idx)
                    tb_tracker.track("loss_entropy", entropy_loss_v, step_idx)
                    tb_tracker.track("loss_policy", loss_policy_v, step_idx)
                    tb_tracker.track("loss_value", loss_value_v, step_idx)
                    tb_tracker.track("loss_total", loss_v, step_idx)
    
    #예외 발생, 게임 해결 시 실행
    finally:
        for p in data_proc_list:
            p.terminate() #자식 process 종료 
            p.join() #동기화
            
            #남은 process가 없도록 
                            

## Result
<img src="./image/data1.png">
<img src="./image/data2.png">
<img src="./image/data3.png">
<img src="./image/data4.png">
<img src="./image/data5.png">


- 수렴 역학 면에서의 결과는 A2C에 병렬환경을 사용한 것과 비슷하지만 속도가 더 빠름 